In [7]:
import sbol2
import pandas as pd 
from openpyxl import Workbook, load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Border, Side
from requests_html import HTMLSession

class seqFile:
    
#     def __init__(self, homeSpace, document, sheet):
    def __init__(self):
        #global varibales for homespace and document 
        self.homeSpace = 'http://sys-bio.org'
        self.document = 'pichia_toolkit_KWK_v002.xml'
        self.sheet = 'ontologies.xlsx'
        
    def roleVariables(self):
        #set Excel file into a dataframe
        df = pd.read_excel('../resources/templates/' + self.sheet, index_col = 0, sheet_name = 1,usecols = [1,2])
        #convert the dataframe into a dictionary
        roleConvertDict = df.to_dict()
        #set dictionary indices and values
        roleName = roleConvertDict['URI']
        #switch indices' and values' postions
        roleDictionary = {uri:role for role, uri in roleName.items()}
        return roleDictionary
    
    def orgVariables(self):
        #set Excel file into a dataframe
        df = pd.read_excel('../resources/templates/' + self.sheet, index_col = 0, sheet_name = 2,usecols = [0,1])
        #convert the dataframe into a dictionary
        organismConvertDict = df.to_dict()
        #set dictionary indices and values
        organismName = organismConvertDict['txid']
        #switch indices' and values' postions
        organismDictionary = {str(txid):organism for organism, txid in organismName.items()}
        return organismDictionary
        
    def inspectDocInfo(self):
        #declare homespace 
        sbol2.setHomespace(self.homeSpace)
        doc = sbol2.Document()
        doc.read('../resources/templates/' + self.document)
#         doc.read(self.document)
        #print document information 
        print(doc) 
    
    def printDocContents(self):
        #declare homespace 
        sbol2.setHomespace(self.homeSpace)
        doc = sbol2.Document()
        doc.read('../resources/templates/' + self.document)
#         doc.read(self.document)
        #print document contents
        for obj in doc:
            print(obj)
    
    def readDocChart(self):
        #declare homespace 
        sbol2.setHomespace(self.homeSpace)
        doc = sbol2.Document()
        doc.read('../resources/templates/' + self.document)
#         doc.read(self.document)
        #create a dictionary to hold all the component defintions' information 
        componentDefinitions = {}
        #iterate through the component definitions 
        roleDict = self.roleVariables()
        orgDict = self.orgVariables()
        for cd in doc.componentDefinitions:
            cdType = cd.type
            #create a dictionary that has a key for the component definition's identity, 
            #and a value for all of its features
            componentFeatures = {}
            persistentIdentity = cd.properties['http://sbols.org/v2#persistentIdentity'][0]
            #iterate through the properties of the component defintions and set them equal to propValue variable 
            for prop in cd.properties:
                try:
                    propValue = cd.properties[prop][0]
                except:
                    propValue = cd.properties[prop]
                    #extract attribute property type
                if propValue == []:
                    propValue = ''
                idx = prop.find('#')
                #if parsing conditions meet, append them into the componentFeatures dictionary as necessary
                if idx >= 1:
                    prop = prop[idx+1:]
                if prop == 'type':
                    prop = 'types'
                if prop == 'http://purl.org/dc/terms/title':
                    prop = 'title'
                if prop == 'http://purl.org/dc/terms/description':
                    prop = 'description'
                if prop == 'http://purl.obolibrary.org/obo/OBI_0001617':
                    prop = 'OBI_0001617'
                propValue = columnMethods(prop, propValue, doc, cdType, roleDict, orgDict).colV
                componentFeatures[prop]= str(propValue)
            #append each componentFeatures dictionary as a value into the componentDefinitions 
            #dictionary with the 'persistentIdentity' serving as the key
            componentDefinitions[persistentIdentity] = componentFeatures
        #return the dictionary of information (temporary, maybe return true if read in correctly)
        return componentDefinitions

    def TEMP_readDocChart1(self):
        #demo of table column names
        columnNames = ['Part Name', 
                'Role', 
                'Design Notes',
               'Altered Sequence',
               'Part Description',
               'Data Source Prefix',
               'Data Source',
               'Source Organism',
               'Target Organism',
               'Circular',
               'length (bp)',
               'Sequence',
               'Data Source',
               'Composite']
        #import dataframe dictionary
        #convert dictionary to dataframe
        df = pd.DataFrame.from_dict(self.readDocChart(), orient = "index")
        #type caste dataframe to a set
        dfSet = set(df)
        #type caste column names to a set
        columnNameOrder = set(columnNames)
        #check difference between the datframe set and the column name order
        dfSetDifference = dfSet.difference(columnNameOrder)
        #check intersection between the datframe set and the column name order
        dfSetIntersection = dfSet.intersection(columnNameOrder)
        #combine the type casted difference and intersection
        finalSetList = list(dfSetIntersection) + list(dfSetDifference)
        #set list to dictionary
        finalSetDictionary = df[finalSetList].to_dict()
        #create dataframe
        dframe = pd.DataFrame.from_dict(finalSetDictionary, orient = "index")
        #return the dataframe, and tranpose the rows and columns
        return dframe.T
    
    def displayDocChart(self):
        return pd.DataFrame.from_dict(self.readDocChart(), orient = "index")
    
    def column_string(self, n):
        string = ""
        while n > 0:
            n, remainder = divmod(n - 1, 26)
            string = chr(65 + remainder) + string
        return string
    
    def returnExcelChart(self):
        wb = load_workbook('../resources/templates/Book4.xlsx')
        ws = wb.active
        df = pd.DataFrame.from_dict(self.readDocChart(), orient = "index")
        ft1 = Font(name = 'Arial', size = 12, color = '548235')
        ft2 = Font(name = 'Calibri', size = 11, bold = True)
        hold = dataframe_to_rows(df, index=False, header=True)
        counter = 0
        ws['A18'].value = ''
        for r in hold:
            if r == [None]:
                continue
            ws.append(r)
            counter += 1
        tab = Table(displayName="Parts_Lib", ref=f"A19:{self.column_string(len(df.columns))}{(len(df) * 2) - 2}")
        style = TableStyleInfo(name="TableStyleLight7", showFirstColumn=False, 
                               showLastColumn=False, showRowStripes=True, showColumnStripes=False)
        cellColor = PatternFill(patternType ='solid', 
                                fgColor = 'DDEBF7')
        cellBorder = Side(border_style='medium', color="000000")
        
        x = df.columns
        cellIndex = len(x)
        for n in range (65, 65 + len(x)):
            alpha = chr(n)
            ws[f'{alpha}19'].fill = cellColor  
            ws[f'{alpha}19'].border = Border(top= cellBorder)
        tab.tableStyleInfo = style
        ws.add_table(tab)
        counter = 0
        for n in range(len(df) - 1, (len(df) * 2 - 1)):
            counter = counter + 1
            for i in ws[n]:
                i.font = ft1
        titleFormat = len(df)
        if titleFormat % 2 > 0:
            titleFormat = titleFormat - 1
        for j in range(19, titleFormat):
            for x in ws[j]:
                x.font = ft2
        wb.save('../outputs/sequenceparts.xlsx')
        wb.close()
    
class columnMethods:
    
    def __init__(self, colN, colV, doc, cdType, roleDict, orgDict):
        self.colN = colN
        self.colV = colV
        self.doc = doc
        self.cdType = cdType
        self.roleDict = roleDict
        self.orgDict = orgDict
        try:
            return getattr(self, self.colN)()
        except AttributeError:
            return getattr(self, 'no_change')()
        
    def no_change(self):
        pass
    
    def role(self):   
        roleVal = str(self.colV)
        if roleVal in self.roleDict:
            self.colV = self.roleDict[roleVal]
            
    def types(self):
         self.colV = self.colV.split('#')[-1]
            
    def sequence(self):
        self.colV = self.doc.getSequence(self.colV).elements
        
    def sourceOrganism(self):
        orgVal = str(self.colV)
        orgVal = orgVal.split('=')[-1]
        txid = self.colV.split('=')[-1]
        if orgVal in self.orgDict:
            self.colV = self.orgDict[orgVal] 
        else:
            session = HTMLSession()
            r = session.get(self.colV)
            v = r.html.find('strong', first = True)
            self.colV = v.text
            self.orgDict[txid] = self.colV

    def targetOrganism(self):
        orgVal = str(self.colV)
        orgVal = orgVal.split('=')[-1]
        txid = self.colV.split('=')[-1]
        if orgVal in self.orgDict:
            self.colV = self.orgDict[orgVal]
        else:
            session = HTMLSession()
            r = session.get(self.colV)
            v = r.html.find('strong', first = True)
            self.colV = v.text
            self.orgDict[txid] = self.colV

In [8]:
# first = seqFile('http://sys-bio.org', 'pichia_toolkit_KWK_v002.xml', 'ontologies.xlsx')
# first = seqFile('http://sys-bio.org', 'pichia_toolkit_KWK_collection.xml', 'ontologies.xlsx')
# first.displayDocChart()
first = seqFile()
first.TEMP_readDocChart1()

,attachment,wasDerivedFrom,types,version,identity,displayId,title,wasGeneratedBy,designNotes,role,targetOrganism,OBI_0001617,sourceOrganism,persistentIdentity,sequence,description
http://examples.org/aMF_916,,https://pubmed.ncbi.nlm.nih.gov/28252957/,DnaRegion,1,http://examples.org/aMF_916/1,aMF_916,aMFΔ,https://synbiohub.org/public/Excel2SBOL/Trunca...,Pre-sequence shortened,signal_peptide_region_of_CDS,Pichia pastoris,28252957,Saccharomyces cerevisiae,http://examples.org/aMF_916,atgagatttcctagtattttcactgctgtgctatttgccgctagtt...,"α-mating factor, pre-sequence shortened"
http://examples.org/Inulinase_u45_aMF_916,,https://pubmed.ncbi.nlm.nih.gov/28252957/,DnaRegion,1,http://examples.org/Inulinase_u45_aMF_916/1,Inulinase_u45_aMF_916,Inulinase-aMFΔ,,NaN,signal_peptide_region_of_CDS,Pichia pastoris,28252957,NaN,http://examples.org/Inulinase_u45_aMF_916,atgaaactggcttactccctgttgctacctctggctggagtttccg...,Inulinase followed by aMFΔ
http://examples.org/Glucoamylase_u45_aMF_916,,https://pubmed.ncbi.nlm.nih.gov/28252957/,DnaRegion,1,http://examples.org/Glucoamylase_u45_aMF_916/1,Glucoamylase_u45_aMF_916,Glucoamylase-aMFΔ,,NaN,signal_peptide_region_of_CDS,Pichia pastoris,28252957,NaN,http://examples.org/Glucoamylase_u45_aMF_916,atgtctttcagatccctattggcattgtcagggttggtctgttctg...,Glucoamylase followed by aMFΔ
http://examples.org/attB,,https://pubmed.ncbi.nlm.nih.gov/28252957/,DnaRegion,1,http://examples.org/attB/1,attB,attB,https://synbiohub.org/public/Excel2SBOL/mutati...,BsaI site removed,site_specific_recombination_target_region,Pichia pastoris,28252957,NaN,http://examples.org/attB,tggccgtggccgtgctcgtcctcgtcggccggcttgtcgacgacgg...,"BxbI recognition site, BsaI site removed"
http://examples.org/aMF_no_EAEA,,https://pubmed.ncbi.nlm.nih.gov/28252957/,DnaRegion,1,http://examples.org/aMF_no_EAEA/1,aMF_no_EAEA,aMF_no_EAEA,https://synbiohub.org/public/Excel2SBOL/Trunca...,No EAEA,signal_peptide_region_of_CDS,Pichia pastoris,28252957,Saccharomyces cerevisiae,http://examples.org/aMF_no_EAEA,atgagatttccttcaatttttactgctgttttattcgcagcatcct...,α-mating factor no EAEA
http://examples.org/yEGFP_Sec,,https://pubmed.ncbi.nlm.nih.gov/28252957/,DnaRegion,1,http://examples.org/yEGFP_Sec/1,yEGFP_Sec,yEGFP_Sec,https://synbiohub.org/public/Excel2SBOL/codon_...,"No start or stop codon, for use with N-termina...",CDS,Pichia pastoris,28252957,Aequorea victoria,http://examples.org/yEGFP_Sec,gtgagcaagggcgaggagctgttcaccggggtggtgcccatcctgg...,Green fluorescent protein
http://examples.org/SA_u45_aMF_916,,https://pubmed.ncbi.nlm.nih.gov/28252957/,DnaRegion,1,http://examples.org/SA_u45_aMF_916/1,SA_u45_aMF_916,SA-aMFΔ,,NaN,signal_peptide_region_of_CDS,Pichia pastoris,28252957,NaN,http://examples.org/SA_u45_aMF_916,atgaagtgggtaactttcatctcattgttattcttgttctcctctg...,Serum albumin followed by aMFΔ
http://examples.org/aMF_916_no_Kex,,https://pubmed.ncbi.nlm.nih.gov/28252957/,DnaRegion,1,http://examples.org/aMF_916_no_Kex/1,aMF_916_no_Kex,aMFΔ_no_Kex,https://synbiohub.org/public/Excel2SBOL/Trunca...,No Kex recognition site,signal_peptide_region_of_CDS,Pichia pastoris,28252957,Saccharomyces cerevisiae,http://examples.org/aMF_916_no_Kex,atgagattcccatcaatttttactgctgttctgttcgccgcttcta...,α-mating factor no Kex recognition site
http://examples.org/pAOX1,,https://pubmed.ncbi.nlm.nih.gov/28252957/,DnaRegion,1,http://examples.org/pAOX1/1,pAOX1,pAOX1,https://synbiohub.org/public/Excel2SBOL/direct/1,NaN,promoter,Pichia pastoris,28252957,Pichia pastoris,http://examples.org/pAOX1,gatctaacatccaaagacgaaaggttgaatgaaacctttttgccat...,Alcohol oxidase 1
http://examples.org/aMF,,https://pubmed.ncbi.nlm.nih.gov/28252957/,DnaRegion,1,http://examples.org/aMF/1,aMF,aMF,https://synbiohub.org/public/Excel2SBOL/mutati...,NaN,signal_peptide_region_of_CDS,Pichia pastoris,28252957,Saccharomyces cerevisiae,http://examples.org/aMF,atgagatttccttcaatttttactgctgttttattcgcagcatcct...,α-mating factor
